## Dataset & augmentation

In [1]:
!pip install wget

In [ ]:
import wget

# The dataset we used belongs to Leaf Segmentation and Counting Challenge (LCC and LSC). 
# It is publicly available at https://www.plant-phenotyping.org/CVPPP2017. 
# The data download links are saved to the dataset_urls.txt file.

train_truth_url, train_url, test_url = open("dataset_urls.txt").readlines()

train_h5 = wget.download(train_url)
train_truth_h5 = wget.download(train_truth_url)
test_h5 = wget.download(test_url)


  2% [.                                                 ]  15327232 / 588219670

In [ ]:
!unzip 'CVPPP2017_training_images.zip'
!unzip 'CVPPP2017_training_truth.zip'
!unzip 'CVPPP2017_testing_images.zip'

In [ ]:
import h5py
import numpy as np

groups = ['A1', 'A2', 'A3']

train_filename = 'CVPPP2017_training_images.h5'
train_imgs = []
with h5py.File(train_filename, "r") as f:
    for group in groups:
        for file_name in list(f[group]):
            train_imgs.append(np.moveaxis(np.array(f[group][file_name]['rgb'])[:, :, :3], [2], [0]))


train_truth_filename = 'CVPPP2017_training_truth.h5'
train_counts = []
train_masks = []
with h5py.File(train_truth_filename, "r") as f:
    for group in groups:
        for file_name in list(f[group]):
            train_counts.append(np.array(f[group][file_name]['count']))
            train_masks.append(np.clip(np.array(f[group][file_name]['label']), 0, 1))

In [ ]:
print(len(train_imgs), len(train_counts), len(train_masks))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

f, axarr = plt.subplots(1,2)

axarr[0].imshow(np.moveaxis(train_imgs[0], [0], [2]))
axarr[1].imshow(train_masks[0], cmap='gray')

f.set_figheight(10)
f.set_figwidth(10)

print('The number of leafs:', train_counts[0])
plt.imshow(train_imgs[0])

In [ ]:
import torch
import torch.nn.functional as F

def pad_to(x, stride):
    h, w = x.shape[-2:]

    if h % stride > 0:
        new_h = h + stride - h % stride
    else:
        new_h = h
    if w % stride > 0:
        new_w = w + stride - w % stride
    else:
        new_w = w

    lh, uh = int((new_h - h) / 2), (new_h - h) - int((new_h - h) / 2)
    lw, uw = int((new_w - w) / 2), (new_w - w) - int((new_w - w) / 2)
    pads = (lw, uw, lh, uh)

    out = F.pad(x, pads, "constant", 0)

    return out, pads

def unpad(x, pad):
    if pad[2] + pad[3] > 0:
        x = x[:, :, pad[2]: -pad[3], :]
    if pad[0] + pad[1] > 0:
        x = x[:, :, :, pad[0]: -pad[1]]
    return x

In [45]:
## Augmentation to be implemented

# Augmentation types from paper:
# (1) Random-Rotate with an interval of 90 to increase the network invariance to slight angular changes; 
# (2) Flip: horizontal, vertical, and horizontal+ vertical; 
# (3) Resize the images to increase the network invariance to different image resolutions; 
# (4) Gamma transform to extend the data by changing the image greyscale; 
# (5) Random-Brightness: the clarity of object depends on scene lighting and camera sensitivity,
# (6) Random change in the contrast range to increase the network invariance to shadows and improve the network performance in low light conditions; 
# (7) Hue Saturation Brightness (HSV): changes in colour channels, degree of lightness or darkness of a colour; and 
# (8) Normalise a characteristic linear transformation which scales a specific range of data values retaining the original data distribution

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import random

# Padding is needed as there will be skip connections concatenations
PAD_BASE = 16

aug_transforms = [
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.RandomApply([transforms.RandomGrayscale(p=0.1)], p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]

aug_transforms = transforms.Compose(aug_transforms + [transforms.ToTensor()])

class LeafDataset(Dataset):
    def __init__(self, imgs, masks, counts, aug_transforms=None):
        super().__init__()
        self.images = imgs
        self.masks = masks
        self.counts = counts

        self.transforms = transforms
    
    def __len__(self):
        return len(self.images)

    def _set_seed(self, seed):
        random.seed(seed)
        torch.manual_seed(seed)
        
    def __getitem__(self, idx):
        seed = random.randint(0, 2**32)
        self._set_seed(seed)
        aug_img = self.transforms(self.images[idx])
        self._set_seed(seed)
        aug_mask = self.transforms(self.masks[idx])
        return pad_to(aug_img, PAD_BASE)[0], self.counts[idx], pad_to(aug_mask, PAD_BASE)[0]


In [52]:
batch_size = 4
train_size = 0.8
dataset_len = len(train_imgs)
train_len = int(dataset_len * train_size)

train_lst = list(zip(train_imgs, train_counts, train_masks))[:train_len]
test_lst= list(zip(train_imgs, train_counts, train_masks))[train_len:]

train_dataset = LeafDataset(train_imgs[:train_len], train_masks[:train_len], train_counts[:train_len], aug_transforms)
test_dataset = LeafDataset(train_imgs[train_len:], train_imgs[train_len:], train_imgs[train_len:], transforms.ToTensor())

train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Build models

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm.notebook import tqdm

In [54]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Working on {device} device')

Working on cpu device


In [55]:
# First stream network

import torchvision.models as models

resnet50 = models.resnet50(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [66]:
for param in resnet50.parameters():
    param.requires_grad = False

# We will stack the 3 channels of the input image with the mask. Thus, 4 channels in total.
resnet50.conv1 = nn.Conv2d(4, 64, kernel_size=(7, 7), stride=2, padding=3, bias=False)
for param in resnet50.conv1.parameters():
    param.requires_grad = True

# We will solve regression problem
resnet50.fc = torch.nn.Linear(2048, 1)
for param in resnet50.fc.parameters():
    param.requires_grad = True

In [68]:
# Second stream network

class UNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.enc_conv1 = nn.Sequential(nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                       nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU())
        self.pool1 = nn.MaxPool2d(2, 2)  # 256 -> 128
        
        self.enc_conv2 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
                                       nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU())
        self.pool2 = nn.MaxPool2d(2, 2)  # 128 -> 64

        self.enc_conv3 = nn.Sequential(nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
                                       nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
                                       nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU())
        self.pool3 = nn.MaxPool2d(2, 2)  # 64 -> 32

        self.enc_conv4 = nn.Sequential(nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
                                       nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
                                       nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU())
        self.pool4 = nn.MaxPool2d(2, 2)  # 32 -> 16

        # bottleneck
        self.bottleneck_conv = nn.Sequential(nn.Conv2d(512, 1024, 3, padding=1), nn.BatchNorm2d(1024), nn.ReLU(),
                                       nn.Conv2d(1024, 1024, 3, padding=1), nn.BatchNorm2d(1024), nn.ReLU())

        # decoder (upsampling)
        self.upsample1 = nn.ConvTranspose2d(1024, 512, 2, stride=2)  # 16 -> 32
        self.dec_conv1 = nn.Sequential(nn.Conv2d(1024, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
                                       nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU())
        
        self.upsample2 = nn.ConvTranspose2d(512, 256, 2, stride=2)  # 16 -> 32
        self.dec_conv2 = nn.Sequential(nn.Conv2d(512, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
                                       nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU())
        
        self.upsample3 = nn.ConvTranspose2d(256, 128, 2, stride=2)  # 32 -> 64
        self.dec_conv3 = nn.Sequential(nn.Conv2d(256, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
                                       nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU())

        self.upsample4 = nn.ConvTranspose2d(128, 64, 2, stride=2)   # 64 -> 128
        self.dec_conv4 = nn.Sequential(nn.Conv2d(128, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                       nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
                                       nn.Conv2d(64, 1, 1))


    def forward(self, x):
        # encoder
        e1 = self.enc_conv1(x)
        e2 = self.enc_conv2(self.pool1(e1))
        e3 = self.enc_conv3(self.pool2(e2))
        e4 = self.enc_conv4(self.pool3(e3))

        # bottleneck
        b = self.bottleneck_conv(self.pool4(e4))

        # decoder
        d1 = self.dec_conv1(torch.concat([e4, self.upsample1(b)], 1))
        d2 = self.dec_conv2(torch.concat([e3, self.upsample2(d1)], 1))
        d3 = self.dec_conv3(torch.concat([e2, self.upsample3(d2)], 1))
        d4 = self.dec_conv4(torch.concat([e1, self.upsample4(d3)], 1))
        return torch.sigmoid(d4)

## Training

In [60]:
def loss_function(y_real, y_pred):
    # dice loss
    num = (y_real * y_pred).view(4, 1, -1).sum(2)
    den = (y_real + y_pred).view(4, 1, -1).sum(2)
    res = -2 * num / den
    dice = res.mean()

    # bce loss
    bce = (-0.5 * y_real * torch.log(y_pred)).sum(2).mean()
    return dice + bce

In [70]:
def train_segmentation(model_segm, model_count, epochs, loss_fn, opt, data_tr, data_val):
    
    train_segm_losses = []
    train_count_losses = []
    val_segm_losses = []
    val_count_losses = []

    for epoch in epochs:
        model_segm.train()
        model_count.train()
        for imgs, counts, masks in tqdm(data_tr):
            imgs, counts, masks = imgs.to(device), counts.to(device), masks.to(device)
            
            # Train segmentation
            opt.zero_grad()
            output = model_segm(imgs)
            loss = loss_fn(output, masks)
            loss.backward()
            opt.step()
            train_segm_losses.append(loss.item())

            # Train counts
            opt.zero_grad()
            output = model_count(torch.stack((imgs, masks)))
            loss = loss_fn(output, counts)
            loss.backward()
            opt.step()
            train_count_losses.append(loss.item())

        
        model_segm.eval()
        model_count.eval()
        with torch.no_grad():        
            for imgs, counts, masks in tqdm(data_val):
                imgs, counts, masks = imgs.to(device), counts.to(device), masks.to(device)
                # Evaluate segmentation
                output = model_segm(imgs)
                loss = loss_fn(output, masks)
                val_segm_losses.append(loss.item())

                # Evaluate counts
                output = model_count(torch.stack((imgs, masks)))
                loss = loss_fn(output, counts)
                val_count_losses.append(loss.item())
        print(f'Epoch: {epoch}, train loss: {train_segm_losses[-1], train_count_losses[-1]}, val loss: {val_segm_losses[-1], val_count_losses[-1]}')


In [ ]:
model_segm = UNet().to(device)
model_count = resnet50.to(device)

In [ ]:
# Training 
epochs = 100
optimizer = optim.Adam(lr=1e-3, weight_decay=1e-4)

train(model_segm, model_count, epochs, loss_function, optimizer, train_dl, test_dl)

## Evaluation

In [ ]:
## eval functions to be added

def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    
    outputs = outputs.squeeze(1).byte()  # BATCH x 1 x H x W => BATCH x H x W
    labels = labels.squeeze(1).byte()
    SMOOTH = 1e-8
    union = (outputs | labels).float().sum((1, 2))         # Will be zero if both are 0
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
    thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return thresholded

## Build web demo with Gradio library

In [ ]:
#!pip install gradio

In [5]:
import gradio as gr
import os


def predict_leaf_number(img):
    # preprocessing = ...
    return model_count(torch.stack((img, model_segm(img))))


with gr.Blocks() as demo:

    gr.Markdown("# Leaf counter")

    with gr.Row():
        im = gr.Image()
        txt = gr.Textbox(value='', label="Leaf number")

    btn = gr.Button(value="Mirror Image")
    btn.click(predict_leaf_number, inputs=[im], outputs=[txt])

    gr.Markdown("## Image Examples")

if __name__ == "__main__":
    demo.launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>